https://www.transtats.bts.gov/Fields.asp?Table_ID=236

In [1]:
from __future__ import division #, print_function # Imports from __future__ since we're running Python 2

In [2]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [29]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 0
%matplotlib inline
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from datetime import datetime

from helpers.feature_engineering import dateStrToDayYear, getMappingOfSimilarCategoricalColumns, \
    compareSimilarCategoricalColumns 
from helpers.my_one_hot_encoder import MyOneHotEncoder
from helpers.py_helpers import is_number
from scipy.stats import skew, kurtosis
from helpers.outliers import MyOutliers
from sklearn.preprocessing import StandardScaler
from helpers.plot_helper import scatter_2d_label
from sklearn.decomposition import PCA # Import the PCA module
from sklearn.manifold import TSNE
from sklearn.decomposition import KernelPCA
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from helpers.performance_issues import subsample_keeping_class_proportions

In [4]:
rng = np.random.RandomState(seed=random_state)

# Flying to New York City - ???

In [5]:
target_col = 'IS_DELAYED'

In [6]:
path_data = os.path.realpath(os.path.join(os.getcwd(), '../Data', 'binary_train_data_numerical_no_outliers.csv'))
assert os.path.isfile(path_data)
path_data

'/home/student/pligor.george@gmail.com/msc_Artificial_Intelligence/dme_Data_Mining/dmedatarats/Data/binary_train_data_numerical_no_outliers.csv'

In [7]:
df = pd.read_csv(path_data, delimiter = ',', header=0)
df.shape

(180133, 533)

In [8]:
df.sample(10, random_state=random_state)

QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK  FL_NUM  DEP_TIME  \
165999        3      7            26            2    2922     520.0   
118690        2      4             3            7    1462     641.0   
170863        1      2             8            1     942    1116.0   
95968         1      3            24            4    3483     643.0   
121221        2      4             9            6     234    2239.0   
151722        2      5             1            7    1485    1955.0   
8018          1      3             5            6    7406    2015.0   
30157         1      1            22            5    4462    2151.0   
59496         2      5            15            7     359    1752.0   
15980         2      6            18            6    3723    1021.0   

        DEP_DELAY  DEP_DELAY_GROUP  DISTANCE  DISTANCE_GROUP     ...      \
165999        0.0              0.0     281.0               2     ...       
118690       11.0              0.0     762.0               4     ...       
170863       -4.0             -1.0    1089.0               5     ...       
95968         3.0              0.0    1381.0               6     ...       
121221       -6.0             -1.0     266.0               2     ...       
151722        0.0              0.0     950.0               4     ...       
8018         -5.0             -1.0     378.0               2     ...       
30157        -6.0             -1.0     282.0               2     ...       
59496        42.0              2.0    2465.0              10     ...       
15980        -4.0             -1.0     289.0               2     ...       

        DEP_TIME_BLK_9  DEP_TIME_BLK_10  DEP_TIME_BLK_11  DEP_TIME_BLK_12  \
165999             0.0              0.0              0.0              0.0   
118690             0.0              0.0              0.0              0.0   
170863             0.0              0.0              0.0              0.0   
95968              0.0              0.0              0.0              0.0   
121221             0.0              0.0              0.0              0.0   
151722             0.0              0.0              0.0              0.0   
8018               0.0              0.0              0.0              0.0   
30157              0.0              0.0              0.0              0.0   
59496              0.0              0.0              1.0              0.0   
15980              0.0              0.0              0.0              0.0   

        DEP_TIME_BLK_13  DEP_TIME_BLK_14  DEP_TIME_BLK_15  DEP_TIME_BLK_16  \
165999              0.0              0.0              0.0              0.0   
118690              0.0              0.0              0.0              0.0   
170863              0.0              0.0              0.0              0.0   
95968               0.0              0.0              0.0              0.0   
121221              0.0              0.0              0.0              1.0   
151722              1.0              0.0              0.0              0.0   
8018                0.0              1.0              0.0              0.0   
30157               0.0              0.0              1.0              0.0   
59496               0.0              0.0              0.0              0.0   
15980               0.0              0.0              0.0              0.0   

        DEP_TIME_BLK_17  IS_DELAYED  
165999              0.0       False  
118690              0.0       False  
170863              0.0        True  
95968               0.0       False  
121221              0.0       False  
151722              0.0       False  
8018                0.0        True  
30157               0.0       False  
59496               0.0        True  
15980               0.0       False  

[10 rows x 533 columns]

In [9]:
df.info()  #here we are verifying that everything is numerical

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180133 entries, 0 to 180132
Columns: 533 entries, QUARTER to IS_DELAYED
dtypes: bool(1), float64(525), int64(7)
memory usage: 731.3 MB


In [10]:
columns_one_hot_encoded = np.array([col for col in df.columns if is_number(col[-1])])
columns_one_hot_encoded

array(['UNIQUE_CARRIER_0', 'UNIQUE_CARRIER_1', 'UNIQUE_CARRIER_2',
       'UNIQUE_CARRIER_3', 'UNIQUE_CARRIER_4', 'UNIQUE_CARRIER_5',
       'UNIQUE_CARRIER_6', 'UNIQUE_CARRIER_7', 'UNIQUE_CARRIER_8',
       'UNIQUE_CARRIER_9', 'UNIQUE_CARRIER_10', 'ORIGIN_0', 'ORIGIN_1',
       'ORIGIN_2', 'ORIGIN_3', 'ORIGIN_4', 'ORIGIN_5', 'ORIGIN_6',
       'ORIGIN_7', 'ORIGIN_8', 'ORIGIN_9', 'ORIGIN_10', 'ORIGIN_11',
       'ORIGIN_12', 'ORIGIN_13', 'ORIGIN_14', 'ORIGIN_15', 'ORIGIN_16',
       'ORIGIN_17', 'ORIGIN_18', 'ORIGIN_19', 'ORIGIN_20', 'ORIGIN_21',
       'ORIGIN_22', 'ORIGIN_23', 'ORIGIN_24', 'ORIGIN_25', 'ORIGIN_26',
       'ORIGIN_27', 'ORIGIN_28', 'ORIGIN_29', 'ORIGIN_30', 'ORIGIN_31',
       'ORIGIN_32', 'ORIGIN_33', 'ORIGIN_34', 'ORIGIN_35', 'ORIGIN_36',
       'ORIGIN_37', 'ORIGIN_38', 'ORIGIN_39', 'ORIGIN_40', 'ORIGIN_41',
       'ORIGIN_42', 'ORIGIN_43', 'ORIGIN_44', 'ORIGIN_45', 'ORIGIN_46',
       'ORIGIN_47', 'ORIGIN_48', 'ORIGIN_49', 'ORIGIN_50', 'ORIGIN_51',
       'ORIGIN_

In [11]:
columns_not_one_hot = np.array([col for col in df.columns if not is_number(col[-1])])
columns_not_one_hot

array(['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_NUM',
       'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_GROUP', 'DISTANCE',
       'DISTANCE_GROUP', 'YDAY', 'IS_DELAYED'], 
      dtype='|S15')

In [12]:
df[columns_not_one_hot].describe()

QUARTER          MONTH   DAY_OF_MONTH    DAY_OF_WEEK  \
count  180133.000000  180133.000000  180133.000000  180133.000000   
mean        2.485286       6.469853      15.779080       3.901384   
std         1.099547       3.376037       8.789477       1.967829   
min         1.000000       1.000000       1.000000       1.000000   
25%         2.000000       4.000000       8.000000       2.000000   
50%         2.000000       6.000000      16.000000       4.000000   
75%         3.000000       9.000000      23.000000       5.000000   
max         4.000000      12.000000      31.000000       7.000000   

              FL_NUM       DEP_TIME      DEP_DELAY  DEP_DELAY_GROUP  \
count  180133.000000  180133.000000  180133.000000    180133.000000   
mean     1656.073862    1404.143255      22.556311         0.982335   
std      1532.067779     518.063078      39.313829         2.671317   
min         1.000000       1.000000     -31.000000        -2.000000   
25%       458.000000     958.000000      -4.000000        -1.000000   
50%      1269.000000    1438.000000       3.000000         0.000000   
75%      2168.000000    1833.000000      37.000000         2.000000   
max      7439.000000    2400.000000     180.000000        12.000000   

            DISTANCE  DISTANCE_GROUP           YDAY  
count  180133.000000   180133.000000  180133.000000  
mean     1052.818340        4.701165     182.171929  
std       699.837374        2.762467     103.605450  
min        94.000000        1.000000       1.000000  
25%       544.000000        3.000000      92.000000  
50%       950.000000        4.000000     182.000000  
75%      1381.000000        6.000000     269.000000  
max      2586.000000       11.000000     366.000000

In [13]:
Xtrain = df.drop(labels=[target_col], axis=1)

In [14]:
ytrain = df[target_col]

In [15]:
columns_to_explore = columns_not_one_hot[columns_not_one_hot != target_col]
#assert (len(columns_to_explore) + 1) == len(columns_not_one_hot)
len(columns_to_explore)

11